In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns



In [4]:
import optuna
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn import model_selection
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, roc_auc_score

In [13]:
q = '''
select *
from workspace.emily_ctr_df
'''
data = pd.read_sql(q, con=bun_dw)
data.to_csv('df_6hr.csv', index=False)

In [5]:
df = df.drop(['imp_content_id', 'imp_content_owner', 'imp_user_id'], axis=1)

In [31]:
df.head()

,label,day,hour,imp_content_position,imp_page_id,imp_ref_page_id,imp_ref_term,imp_ref_source,user_sex,user_age,...,owner_item_count,owner_interest,owner_follower_cnt,owner_bunpay_count,content_price,content_category_id,content_emergency_cnt,content_comment_cnt,content_interest,content_pfavcnt
0,0,20200601,1,2808,1,0,13769,1,2,27,...,941,106,0,0,23000,57,0,0,268,12
1,0,20200601,1,3652,1,0,9593,1,2,26,...,1541,25831,0,179,31000,31,1,0,800,19
2,0,20200601,1,1699,1,0,9339,0,2,27,...,154,198,0,0,15000,8,1,1,4198,148
3,0,20200601,1,2809,1,0,6007,1,1,27,...,1013,12202,0,22,13500,8,0,0,3929,448
4,0,20200601,1,1699,1,0,6007,1,1,51,...,461,178535,0,4,12000,58,0,0,1636,107


In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

sparse_features = ['hour', 'imp_content_position', 'imp_page_id', 'imp_ref_page_id', 'imp_ref_term', 'imp_ref_source', 'user_sex', 'content_category_id']
for f in sparse_features:
    df[f] = df[f].fillna('-1')
    df[f] = le.fit_transform(df[f])

In [35]:
from sklearn.model_selection import train_test_split


x = df.iloc[:, 1:]
y = df.iloc[:, 0]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [43]:
lr = LogisticRegression(class_weight='balanced', n_jobs=-1, random_state=42)
lr.fit(x_train, y_train)
pred = lr.predict(x_test)
print('log loss:', log_loss(y_test.astype(int), pred.astype(int)))

log loss: 8.651399912216744


In [47]:
# class_weight: dict or ‘balanced’, default=None
import random
import math
lr = LogisticRegression(class_weight={0: 1, 1: math.log(random.randint(20, 80))}, n_jobs=-1, random_state=42)
lr.fit(x_train, y_train)
pred = lr.predict(x_test)
print(len([i for i in pred if i > 0.5])/len(pred))
# from sklearn.metrics import log_loss

print('log loss:', log_loss(y_test.astype(int), pred.astype(int)))

0.0
log loss: 1.0303491137269998


In [41]:
print(len([i for i in y_train if i == 1])/len(y_train))

0.02916155031513786


In [42]:
print(len([i for i in y_test if i == 1])/len(y_test))

0.029831662301702776


In [ ]:
class_weight={0: 1, 1: lr_class_weight}